In [ ]:
#r "nuget: FParsec, 1.1.1"

open FParsec
open System.IO
open System.Text



Installed Packages FParsec, 1.1.1

In [ ]:
// PARSERS for input

let pInt = pint32 |>> int
let pTuple = (pInt .>> pstring ",") .>>. pInt
let pLine = (pTuple .>> pstring " -> ") .>>. pTuple
let pAll = sepBy pLine newline

// HELPERS FOR CALCS
let diagonal (p1, p2) = fst p1 <> fst p2 && snd p1 <> snd p2

// Equation of a line. Shoutout to highschool algeometry.
let eoal x1 y1 x2 y2 x =
    let m = (y2 - y1) / (x2 - x1)
    let b = y1 - (m * x1)
    (m * x) + b

// We have to treat vertical lines special because VERTICAL LINES ARE NOT FUNCTIONS
let getPoints = function
| (x1, y1), (x2, y2) when x1 = x2 -> seq { for i in y1..sign(y2 - y1)..y2 do x1, i }
| (x1, y1), (x2, y2) -> seq { for i in x1..sign(x2 - x1)..x2 do i, eoal x1 y1 x2 y2 i }

// The solver
let solve =
    Seq.collect getPoints
    >> Seq.countBy id
    >> Seq.filter (snd >> ((<=) 2))
    >> Seq.length

// The data
let data =
    match runParserOnFile pAll () "day5.txt" Encoding.ASCII with
    | Success (result, _, _) -> result
    | _ -> failwith "oops!"

let day5part1solution = solve <| Seq.filter (not << diagonal) data
let day5part2solution = solve data

day5part1solution, day5part2solution

Item1,Item2
5145,16518


In [ ]:
[0,9,5,9;8,0,0,8;9,4,3,4;2,2,2,1;7,0,7,4;6,4,2,0;0,9,2,9;3,4,1,4;0,0,8,8;5,5,8,2]
|> Seq.collect 
    (fun(x1,y1,x2,y2)->
        let xGradient,yGradient=sign(x2-x1),sign(y2-y1)
        in 
        Seq.unfold 
            (fun(x,y)-> if (x,y)<>(x2,y2) then Some ((x+xGradient,y+yGradient),(x+xGradient,y+yGradient)) else None) 
            (x1-xGradient,y1-yGradient)
    )
|> Seq.countBy id
|> Seq.sumBy (snd >> (+)(-1) >> sign >> max 0)